In [266]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

Необходимо получить список фильмов из рейтинга IMDb Топ-250 со странцы https://www.imdb.com/chart/top
Для того, чтобы была возможность отбирать фильмы по жанру, году и т.д. необходимо распрсить таблицу Топ-250 и сформировать таблицу фильмов с их характеристиками.

# Получение списка фильмов рейтинга Топ-250 IMDb
https://www.imdb.com/chart/top

In [427]:
domain = 'https://www.imdb.com' # домен сайта IMDb
top = '/chart/top' # страница рейтинга Топ-250
url = domain + top 

### Парсинг страницы Топ-250

In [204]:
req_text = requests.get(url).text

In [206]:
soup = BeautifulSoup(req_text, 'html.parser')

### Получение контента таблицы Топ-250

In [526]:
top_250 = []
movies = []

# получение таблицы с фильмами со страницы сайта
rows = soup.tbody.find_all('tr', 
                           #limit=2 # через limit можно ограничить количество загружаемых строк
                          )
for row in rows:
    film = row.find(name='td', attrs={'class': 'titleColumn'}).text.split('\n')
    
    no = film[1].strip('. ') # номер в рейтинге
    movie_id = re.sub('/title/', '', row.td.a['href']).strip('/') # id вырезан из ссылки на фильм
    year = film[3].strip('()') # год выпуска
    rating = row.find(name='td', attrs={'class': 'ratingColumn imdbRating'}).text.strip()
    
    top_250.append([no, movie_id, year, rating])

#### DataFrame с таблицей Топ-250
Номер, год выпуска, рейтинг - приводятся к числовому формату.

In [528]:
top_rated_movies = pd.DataFrame(data=top_250, columns=['no', 'id', 'year', 'rating'])

In [529]:
top_rated_movies['no'] = top_rated_movies['no'].astype('int32')
top_rated_movies['year'] = top_rated_movies['year'].astype('int32')
top_rated_movies['rating'] = top_rated_movies['rating'].astype('float32')

In [530]:
top_rated_movies.tail()

,no,id,year,rating
245,246,tt0058946,1966,8.0
246,247,tt0367110,2004,8.0
247,248,tt0050613,1957,8.0
248,249,tt0242519,2000,8.0
249,250,tt0169102,2001,8.0


In [536]:
# сохранение Топ-250 в файл
top_rated_movies.to_csv('top_rated_movies')

# Получение данных о фильмах

#### DataFrame с таблицей Фильмов

In [531]:
top_movies = pd.DataFrame(data=movies, columns=['name','genre', 'length', 'age'])

In [532]:
top_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    250 non-null    object
 1   genre   250 non-null    object
 2   length  250 non-null    object
 3   age     250 non-null    object
dtypes: object(4)
memory usage: 7.9+ KB


In [533]:
top_movies['age'].value_counts()

16+         51
18+         34
12+         25
0+          19
R           17
            ..
2h 41min     1
2h 25min     1
1h 38min     1
2h 43min     1
1h 52min     1
Name: age, Length: 63, dtype: int64

Найти год, в который были выпущены самые рейтинговые фильмы.

In [161]:
year_top = top_rated_mevies.groupby(by='year')['name'].count().sort_values(ascending=False)

Найти все фильмы "Властелин колец" и "Звездные войны".
Найти есть ли в рейтинге фильмы о Гарри Поттере.

----

In [441]:
row = rows[0]
movie_link = row.td.a['href']
movie_link

'/title/tt0111161/'

In [442]:
re.sub('/title/', '', movie_link).strip('/')

'tt0111161'

In [443]:
re.sub('/title/', '', row.td.a['href']).strip('/')

'tt0111161'

# Формирование таблицы фильмов
https://www.imdb.com/title/tt0111161

Добавить информацию о жанрах.
Сформировать таблицу с информацие о фильме: дата выхода, возрастной ценз, жанр, длительность.

Список характеристик фильма

In [488]:
movie_description = []

In [489]:
movie_id = 'tt0245429'

Функция для получения характеристик фильма

In [512]:
def get_movie_description(movie_id):    
    url_movie_overview = '{}/title/{}/'.format(domain, movie_id)
    soup_movie_overview = BeautifulSoup(requests.get(url_movie_overview).text)
    
    wrapper_list = soup_movie_overview.find(name='div', attrs={'class': 'subtext'}).text.split('|')

    name = get_name(soup_movie_overview)
    genre = get_genre(wrapper_list)
    length = get_length(wrapper_list)
    age = get_age(wrapper_list)
    
    return [movie_id, name, genre, age]

In [513]:
get_movie_description(movie_id)

['tt0245429',
 'Унесённые призраками',
 ['Animation', 'Adventure', 'Family'],
 '12+']

----

Парсинг страницы с описанием фильма

In [460]:
url_movie_overview = '{}/title/{}/'.format(domain, movie_id)
url_movie_overview

'https://www.imdb.com/title/tt0245429/'

In [461]:
movie_overview = requests.get(url_movie_overview).text

In [462]:
soup_movie_overview = BeautifulSoup(movie_overview)

----
Получение наименования фильма

In [481]:
def get_name(soup_movie_overview):
    movie_title = soup_movie_overview.find(name='div', attrs={'class': 'title_wrapper'}).h1.text
    movie_title = re.search(r'.*\xa0', movie_title).group().strip('\xa0')
    return movie_title

In [463]:
movie_title = soup_movie_overview.find(name='div', attrs={'class': 'title_wrapper'}).h1.text

In [464]:
movie_name = re.search(r'.*\xa0', movie_title).group().strip('\xa0')
movie_name

'Унесённые призраками'

Получение данных о жанре, возрасном ограничении, длительности.

In [400]:
title_wrapper = soup_movie_overview.find(name='div', attrs={'class': 'subtext'})
title_wrapper

<div class="subtext">
                    0+
    <span class="ghost">|</span> <time datetime="PT116M">
                        1h 56min
                    </time>
<span class="ghost">|</span>
<a href="/search/title?genres=comedy&amp;explore=title_type,genres">Comedy</a>, 
<a href="/search/title?genres=drama&amp;explore=title_type,genres">Drama</a>, 
<a href="/search/title?genres=romance&amp;explore=title_type,genres">Romance</a>
<span class="ghost">|</span>
<a href="/title/tt0118799/releaseinfo" title="See more release dates">16 July 1999 (Russia)
</a> </div>

In [401]:
wrapper_list = title_wrapper.text.split('|')
wrapper_list

['\n                    0+\n    ',
 ' \n                        1h 56min\n                    \n',
 '\nComedy, \nDrama, \nRomance\n',
 '\n16 July 1999 (Russia)\n ']

Жанр

In [502]:
def get_genre(wrapper_list):
    genre_list = wrapper_list[2].strip('\n')
    genre_list_split = re.findall('\w*', genre_list)
    genre = list(filter(None, genre_list_split))
    return genre

In [402]:
wrapper_list[2]

'\nComedy, \nDrama, \nRomance\n'

In [403]:
genre_list = wrapper_list[2].strip('\n')
genre_list_split = re.findall('\w*', genre_list)
genre = list(filter(None, genre_list_split))
genre

['Comedy', 'Drama', 'Romance']

In [404]:
movie_description.append(genre)

Длительность

In [506]:
def get_length(wrapper_list):
    return wrapper_list[1].strip('\n ')

In [507]:
get_length(wrapper_list)

'1h 56min'

In [412]:
length = wrapper_list[1].strip('\n ')
length

'1h 56min'

In [413]:
movie_description.append(length)

Возрастное ограничение

In [510]:
def get_age(wrapper_list):
    return wrapper_list[0].strip('\n ')

In [423]:
age = wrapper_list[0].strip('\n ')
age

'0+'

In [425]:
movie_description.append(age)

DataFrame

In [418]:
length_format = '%Hh %Mmin'

In [414]:
movie_description

['Жизнь прекрасна', ['Comedy', 'Drama', 'Romance'], '1h 56min']

In [511]:
movies = []
movies.append(movie_description)

[['Унесённые призраками']]

In [426]:
df = pd.DataFrame(data=movies, columns=['name','genre', 'length', 'age'])
df

,name,genre,length,age
0,Жизнь прекрасна,"[Comedy, Drama, Romance]",1h 56min,0+


In [419]:
pd.to_datetime(df['length'], format=length_format)

0   1900-01-01 01:56:00
Name: length, dtype: datetime64[ns]

In [353]:
genre_list_split

['Crime', '', '', 'Drama', '']

In [293]:
description = []
for element in wrapper:
    if element.strip(' |'): description.append(element.strip(' |'))
description    

['16+', '1h 36min', 'Crime,', 'Drama', 'November 1961 (Soviet Union)']

In [244]:
map(lambda x: print(x), wrapper)

In [255]:
regimentNames = ['Night Riflemen', 'Jungle Scouts', 'The Dragoons', 'Midnight Revengence', 'Wily Warriors']
capitalizer = lambda x: x.upper()
regimentNamesCapitalized_m = list(map(capitalizer, regimentNames))
regimentNamesCapitalized_m

['NIGHT RIFLEMEN',
 'JUNGLE SCOUTS',
 'THE DRAGOONS',
 'MIDNIGHT REVENGENCE',
 'WILY WARRIORS']

In [165]:
top_rated_mevies.loc[top_rated_mevies['year'] == 1995]

,no,name,year,rating
19,20,Семь,1995,8.6
32,33,Подозрительные лица,1995,8.5
77,78,Храброе сердце,1995,8.3
80,81,История игрушек,1995,8.3
123,124,Схватка,1995,8.2
140,141,Казино,1995,8.2
192,193,Перед рассветом,1995,8.1
230,231,Ненависть,1995,8.0


In [149]:
top_rated_mevies.corr()

,no,year,rating
no,1.000000,-0.007788,-0.906663
year,-0.007788,1.000000,0.027455
rating,-0.906663,0.027455,1.000000


In [583]:
income = 604000

In [584]:
ss = 34500

In [585]:
zp = 63201

In [586]:
rent = 32945

In [587]:
tax = income*6/100
tax

36240.0

In [588]:
cred = 230550

In [589]:
(income)-(ss+zp+rent)-tax-cred

206564.0

In [595]:
invest = 10

In [596]:
income = 2.5

In [597]:
(income - invest)/invest

-0.75

In [594]:
(200 - 1000)/1000

-0.8

In [598]:
expence = 2500

In [599]:
income = 5*400

In [ ]:
marge = 1000

In [601]:
150/100*100

150.0

In [602]:
1000/2500*100

40.0

In [604]:
555/22755*100

2.4390243902439024

In [605]:
conv = 2.154

In [606]:
income = 21824

In [608]:
client = income*conv/100
client

470.08896

In [609]:
conv = 1.821

In [610]:
client = 712

In [611]:
people = client/conv*100

In [612]:
people

39099.39593629874

In [613]:
main = 10000

In [614]:
goods = 8000

In [615]:
cart = 3000

In [616]:
form = 1000

In [617]:
order = 100

In [618]:
conv = []

In [620]:
step1 = [1, main/main, main/main]
conv.append(step1)

In [622]:
step2 = [2, goods/main, goods/main]
conv.append(step2)

In [623]:
step3 = [3, cart/main, cart/goods]
conv.append(step3)

In [624]:
step4 = [4, form/main, form/cart]
conv.append(step4)

In [625]:
step5 = [5, order/main, order/form]
conv.append(step5)

In [630]:
df = pd.DataFrame(data=conv)
df.iloc[:, [1,2]].transform(lambda x: x*100)

,1,2
0,100.0,100.000000
1,80.0,80.000000
2,30.0,37.500000
3,10.0,33.333333
4,1.0,10.000000
